In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.datasets import load_digits
import time
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from ipywidgets import interact
import pickle
import time
from sklearn.decomposition import PCA

In [2]:
data = pd.read_json('yelp_academic_dataset_business.json', lines=True, orient='columns', encoding='utf-8')

In [3]:
df1 = data.mask(data.astype(object).eq('None')).dropna()

df2 = pd.json_normalize(df1['attributes'])
# df3 = pd.json_normalize(df1['hours'])

df1 = df1.drop(["attributes", "hours"], axis = 1)

df_concat = pd.concat([df1, df2], axis=1)

In [4]:
df_concat['categories'] = df_concat['categories'].str.lower()
df = df_concat.loc[df_concat.categories.str.contains("restaurant|food|deli|pizza|cafe", na=False)]

In [5]:
df = df.replace(['',' ','NULL'],np.nan)

In [6]:
temp = df.dropna(thresh=df.shape[0]*0.4,how='all',axis=1)

In [7]:
temp = temp.dropna(how='any')

In [8]:
temp = temp.drop(["business_id", "name", "address", "categories", "BusinessParking", "city"], axis = 1)
temp = temp[temp.postal_code.apply(lambda x: x.isnumeric())]

In [9]:
from sklearn.feature_extraction import FeatureHasher
fh = FeatureHasher(n_features=6, input_type='string')
hashed_features = fh.fit_transform(temp['state'])
dfx = pd.DataFrame(hashed_features.toarray())
dfx.columns = ['state_0', 'state_1', 'state_2', 'state_3', 'state_4', 'state_5']
hashed = pd.concat([temp.reset_index(drop=True), dfx.reset_index(drop=True)], axis=1)
hashed

,state,postal_code,latitude,longitude,stars,review_count,is_open,BikeParking,BusinessAcceptsCreditCards,RestaurantsPriceRange2,state_0,state_1,state_2,state_3,state_4,state_5
0,CO,80302,40.017544,-105.283348,4.0,86.0,1.0,True,True,2,0.0,0.0,0.0,-1.0,1.0,0.0
1,OR,97218,45.588906,-122.593331,4.0,126.0,1.0,False,True,2,0.0,0.0,0.0,1.0,1.0,0.0
2,CO,80302,39.998449,-105.281006,3.5,5.0,1.0,True,True,2,0.0,0.0,0.0,-1.0,1.0,0.0
3,MA,01960,42.541155,-70.973438,4.0,39.0,1.0,True,True,2,0.0,0.0,0.0,0.0,2.0,0.0
4,MA,02128,42.363442,-71.025781,3.5,856.0,1.0,True,True,2,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14816,OR,97005,45.492267,-122.787117,3.5,197.0,0.0,True,True,1,0.0,0.0,0.0,1.0,1.0,0.0
14817,OR,97213,45.536427,-122.618696,3.5,59.0,1.0,True,True,2,0.0,0.0,0.0,1.0,1.0,0.0
14818,MA,02145,42.395540,-71.079465,3.0,14.0,1.0,False,True,4,0.0,0.0,0.0,0.0,2.0,0.0
14819,MA,02120,42.332385,-71.099752,4.5,105.0,0.0,True,True,2,0.0,0.0,0.0,0.0,2.0,0.0


In [10]:
hashed = hashed.drop("state", axis=1)
d = {"True" : 1, "False" : 0}
hashed["BikeParking"] = hashed["BikeParking"].replace(d)
hashed["BusinessAcceptsCreditCards"] = hashed["BusinessAcceptsCreditCards"].replace(d)
hashed = hashed.replace({'None' : 0})
hashed

,postal_code,latitude,longitude,stars,review_count,is_open,BikeParking,BusinessAcceptsCreditCards,RestaurantsPriceRange2,state_0,state_1,state_2,state_3,state_4,state_5
0,80302,40.017544,-105.283348,4.0,86.0,1.0,1,1,2,0.0,0.0,0.0,-1.0,1.0,0.0
1,97218,45.588906,-122.593331,4.0,126.0,1.0,0,1,2,0.0,0.0,0.0,1.0,1.0,0.0
2,80302,39.998449,-105.281006,3.5,5.0,1.0,1,1,2,0.0,0.0,0.0,-1.0,1.0,0.0
3,01960,42.541155,-70.973438,4.0,39.0,1.0,1,1,2,0.0,0.0,0.0,0.0,2.0,0.0
4,02128,42.363442,-71.025781,3.5,856.0,1.0,1,1,2,0.0,0.0,0.0,0.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14816,97005,45.492267,-122.787117,3.5,197.0,0.0,1,1,1,0.0,0.0,0.0,1.0,1.0,0.0
14817,97213,45.536427,-122.618696,3.5,59.0,1.0,1,1,2,0.0,0.0,0.0,1.0,1.0,0.0
14818,02145,42.395540,-71.079465,3.0,14.0,1.0,0,1,4,0.0,0.0,0.0,0.0,2.0,0.0
14819,02120,42.332385,-71.099752,4.5,105.0,0.0,1,1,2,0.0,0.0,0.0,0.0,2.0,0.0


In [11]:
feature_vectors = hashed.drop("is_open", axis=1)
class_labels = hashed["is_open"]
train_vectors, test_vectors, train_labels, test_labels = train_test_split(feature_vectors, class_labels,random_state=202, test_size=0.25)

In [12]:
n_components = 1 # This is much less than the original n_features

print("Extracting the top %d eigenfaces from %d faces" % (n_components, train_vectors.shape[0]))

#Set up the pca object with the number of compoents we want to find
pca = PCA(n_components=n_components, whiten=True)

#Fit the training data to the pca model.
_ = pca.fit(train_vectors)

total_variance = np.sum(pca.explained_variance_ratio_)*100
print("These %d eigenvectors account for a total of %d percent of the total variance in the original dataset"
      % (n_components, total_variance))


pca_train_vectors = pca.transform(train_vectors)
pca_test_vectors = pca.transform(test_vectors)

print("Training set changed from a size of: ", train_vectors.shape, ' to: ', pca_train_vectors.shape)
print("Testing set changed from a size of: ", test_vectors.shape, ' to: ', pca_test_vectors.shape)

Extracting the top 1 eigenfaces from 11115 faces
These 1 eigenvectors account for a total of 99 percent of the total variance in the original dataset
Training set changed from a size of:  (11115, 14)  to:  (11115, 1)
Testing set changed from a size of:  (3706, 14)  to:  (3706, 1)


In [13]:
pca_train_vectors

array([[ 0.94925844],
       [ 0.94888355],
       [-0.07643797],
       ...,
       [ 1.47582896],
       [-0.37586746],
       [-0.37823383]])

In [ ]:
filename = 'covid_business_pca.p'

start = time.time()

#make some temporary variables so you can change this easily
tmp_vectors_90 = pca_train_vectors
tmp_labels_90 = train_labels

print("Fitting the classifier to the training set")
param_grid = {'C': [1e3, 5e3, 1e4],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01],
               'kernel' : ['linear']}
clf_90 = GridSearchCV(SVC(class_weight='balanced'), param_grid, n_jobs=-1)
clf_90 = clf_90.fit(tmp_vectors_90, tmp_labels_90)
print("Best estimator found by grid search:")
print(clf_90.best_estimator_)

end = time.time()
print("Runtime",end - start)

pickle.dump(clf_90, open(filename, 'wb'))

Fitting the classifier to the training set


In [26]:
predict_vectors_90 = pca_test_vectors
true_labels_90 = test_labels

print("Predicting if the business is open on test set")
pred_labels_90 = clf_90.predict(predict_vectors_90)

print(classification_report(true_labels_90, pred_labels_90))
print(confusion_matrix(true_labels_90, pred_labels_90, labels=range(2)))

Predicting people's names on the test set
              precision    recall  f1-score   support

         0.0       0.36      0.39      0.37      1171
         1.0       0.70      0.68      0.69      2535

    accuracy                           0.58      3706
   macro avg       0.53      0.53      0.53      3706
weighted avg       0.59      0.58      0.59      3706

[[ 451  720]
 [ 818 1717]]
